In [1]:
# pip install torch transformers datasets accelerate gradio

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


C:\Users\begat\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
def chat_with_bot(user_input):
    # Set the pad_token to the same as the eos_token
    tokenizer.pad_token = tokenizer.eos_token
    
    # Encode the input and create an attention mask
    inputs = tokenizer.encode_plus(user_input + tokenizer.eos_token, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    # Generate a response from the model
    response_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=100, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    return response

print(chat_with_bot("What is the best opening in chess?"))
print



What is the best opening in chess?


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [4]:
import json
from datasets import Dataset

# Load the first JSON file
with open("chess_com_qa.json", "r") as f1:
    data1 = json.load(f1)

# Load the second JSON file
with open("chess_qa.json", "r") as f2:
    data2 = json.load(f2)

# Combine the data from both files
combined_data = data1 + data2  # Assuming both files contain lists of dictionaries

# Inspect the structure of the combined data
print(type(combined_data))  # Should print the type of data (list or dict)
print(combined_data[:1])    # Print the first element to see its structure

# Assuming combined_data is a list of dictionaries with "question" and "answer" keys
chat_data = {"input_text": [], "response_text": []}
for conv in combined_data:
    chat_data["input_text"].append(conv["question"])
    chat_data["response_text"].append(conv["answer"])

# Convert to dataset format
dataset = Dataset.from_dict(chat_data)

print(dataset)


<class 'list'>
[{'question': 'What is a more efficient thinking process?', 'answer': "I'm a bit of a perfectionist and strive to make the best move every game. As a result I think I expend too much brain power throughout the game and get burned out or overwhelmed by the possibilites, for example in the opening I would try consider all my opponents responses of which there are many in the opening, sometimes I feel habits such as these are unnecessary and are just a waste of energy. What is the most correct way to expend your mental energy in a chess game?"}]
Dataset({
    features: ['input_text', 'response_text'],
    num_rows: 700
})


In [5]:
def tokenize_function(samples):
    inputs = tokenizer(samples["input_text"], padding="max_length", truncation=True, max_length=128)
    responses = tokenizer(samples["response_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = responses["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./chess-chatbot",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
#     evaluation_strategy="epoch",
    evaluation_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()
model.save_pretrained("./fine_tuned_chess_chatbot")
tokenizer.save_pretrained("./fine_tuned_chess_chatbot")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,3.689000


('./fine_tuned_chess_chatbot\\tokenizer_config.json',
 './fine_tuned_chess_chatbot\\special_tokens_map.json',
 './fine_tuned_chess_chatbot\\vocab.json',
 './fine_tuned_chess_chatbot\\merges.txt',
 './fine_tuned_chess_chatbot\\added_tokens.json',
 './fine_tuned_chess_chatbot\\tokenizer.json')

In [10]:
# Load fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_chess_chatbot")
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_chess_chatbot")

print('model done successfully')

model done successfully


In [18]:
def chat_with_bot(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=50)
    outputs = model.generate(
        **inputs,
        max_length=150,  # Increase max length for more detailed responses
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,  # Adjust temperature for more or less creative responses
        repetition_penalty=2.0,  # Penalize repetitive sequences
        num_return_sequences=1,
        no_repeat_ngram_size=1,
        early_stopping=True
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

print(chat_with_bot("Is it ever advantageous to make a trade that loses material for the sake a positional advantage?"))


Is it ever advantageous to make a trade that loses material for the sake a positional advantage?
